# Base Dados IBGE

Os códigos discriminados nesse notebooke dizem repeito a um projeto envolvendo a base de dados do **IBGE(Instituto Brasileiro de Geografia e Estatística)**

## Conteúdo de Apoio
<b>Livro:</b> Python para Analise de Dados - Tratamento de dados com Pandas, Numpy e IPython
<br><b>Autor:</b> Wes McKinney</br>
<br><b>Editora:</b> O'Reilly</br>

<br></br><b>Livro:</b> Python Escreva seus primeiros programas
<br><b>Autor:</b> Felipe Cruz</br>
<br><b>Editora:</b> Casa do Código</br>



## Informações

<b>Fonte:</b> https://servicodados.ibge.gov.br/api/docs

## Importações

In [53]:
import pandas as pd
import json
import requests
import os

## Funções de Auxilio

In [2]:
def fn_busca_lista_dados_IBGE(url):
    """
    Função
        Essa função é reponsável por trazer uma lista de informações do site do IBGE, onde passando a URL da API,
        ela retornara as informações desejadas
        
        Args:
            String: Campo destinado a url do IBGE, mais especificamente a API que será utilizada
        Return:
            Int = codigo do status da requisição, isto é se foi bem sucedida ou não, nesse caso 
                200 para bem sucedida.
            List/ None = representa a lista com os dicionarios de informações fornecidos pelo IBGE
    """
    response = requests.request("GET", url)
    json_response = json.loads(response.content)
    if response.status_code == 200:
        return response.status_code, json_response
    return response.status_code, None

In [50]:
def fn_criar_pasta(caminho):
    """
    Função
        Essa função é reponsável por criar uma pasta conforme caminho passado
        
        Args:
            String: Campo destinado ao caminho onde essa pasta será criada
        Return:
            Int = 0 - Falha, 1 - Sucesso
            String = Mensagem
    """
    resultado = 0
    try:
        os.makedirs(pasta_arquivos_csv)
        mensagem = "O diretório {} foi criado com sucesso.".format(caminho)
        resultado = 1
    except FileExistsError:
        mensagem = "O diretorio {} já existe".format(caminho)
    except OSError:
        mensagem = "Ocorreu um erro na criacao do diretorio: {}".format(caminho)
    
    return resultado, mensagem

## Estados(UF) da Federação

É possivel obtêlos atravé do site: https://servicodados.ibge.gov.br/api/docs/localidades?versao=1#api-UFs-estadosGet onde esta a API utilizada para trazer os estados

### Preparando as listas de dados para os DataFrames

In [3]:
url = "https://servicodados.ibge.gov.br/api/v1/localidades/estados"
http_retorno, json_retorno = fn_busca_lista_dados_IBGE(url)

In [4]:
# Realizar uma cópia da lista trazida, armazenando os estados na lista 'estados'
if len(json_retorno) > 0:
    estados = json_retorno.copy() 

<b> OBS. : </b>
Foi identificado que a API de Estados/ UFs, trazidas pelo IBGE,  nos devolve uma lista de arquivos JSON, que no Python é interpretado como um dicionário, sendo assim, cada estado será um dicionario.

Um outro ponto que nos é válido ressaltar é que cada dicionario é acompanhado por um outro dicionario, com informações referentes ao estado, como Região do Pais, sendo assim essa informação sera aramzenada em uma outra lista e depois ambas serão vinculadas para que seja gerado um arquivo .CSV mais congruente.

In [5]:
# Lista com as informações das regiões dos estados
regioes_por_estados = [es["regiao"] for es in estados]

# Criar uma nova chave nos dicionarios dos estados que faça referencia a região deles
for es in estados:
    es.update( {"id_regiao":es['regiao']['id']})    

### Criando os Dataframes

In [6]:
# DataFrame das Regiões
df_regioes_por_estados = pd.DataFrame(regioes_por_estados)
df_regioes_por_estados.rename(columns = {'id' : 'id_regiao',
                                         'nome': 'nome_regiao',
                                         'sigla': 'sigla_regiao'},
                             inplace = True)

In [7]:
# DataFrame Auxiliar com os Estados
df_estados_aux = pd.DataFrame(estados, 
                          columns = ['id', 'sigla', 'nome', 'id_regiao'])
df_estados_aux.rename(columns = {'id' : 'id_estado',
                                 'nome': 'nome_estado',
                                 'sigla': 'sigla_estado'},
                             inplace = True)

In [8]:
# Gerando o DataFrame com as informações dos Estados
df_estados = pd.concat([df_regioes_por_estados, df_estados_aux], axis = 1)

In [9]:
df_estados.head(3)

,id_regiao,nome_regiao,sigla_regiao,id_estado,sigla_estado,nome_estado,id_regiao
0,1,Norte,N,11,RO,Rondônia,1
1,1,Norte,N,12,AC,Acre,1
2,1,Norte,N,13,AM,Amazonas,1


## Municipios Brasileiros

É possivel obtêlos atravé do site: https://servicodados.ibge.gov.br/api/docs/localidades?versao=1#api-Municipios-estadosUFMunicipiosGet onde esta a API utilizada para trazer os estados

### Preparando as listas de dados

In [10]:
url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
http_retorno, json_retorno = fn_busca_lista_dados_IBGE(url)

In [11]:
# Realizar uma cópia da lista trazida, armazenando os estados na lista 'estados'
if len(json_retorno) > 0:
    municipios = json_retorno.copy() 

In [12]:
# Extrair as chaves correspondentes a microregiao, mesoregiao, UF, regiao
ls_microregiao = [mi['microrregiao'] for mi in municipios]
ls_mesoregiao = [me['mesorregiao'] for me in ls_microregiao]

# Criar uma nova chave nos dicionarios das mesoregioes que façam referencia aos estados
for ms in ls_mesoregiao:
    ms.update( {"id_estado":ms['UF']['id']})     
    
# Criar uma nova chave nos dicionarios das microregioes que façam referencia as mesoregioes
for me in ls_microregiao:
    me.update( {"id_mesoregiao":me['mesorregiao']['id']})   
    
#
for mu in municipios:
    mu.update( {"id_microrregiao":mu['microrregiao']['id']})   


### Criando os DataFrames

In [13]:
# DataFrame das MesoRegiões
df_mesoregioes = pd.DataFrame(ls_mesoregiao, 
                              columns = ['id', 'nome', 'id_estado'])
df_mesoregioes.rename(columns = {'id' : 'id_mesoregiao',
                                 'nome': 'nome_mesoregiao'},
                                 inplace = True)

In [14]:
df_mesoregioes.head()

,id_mesoregiao,nome_mesoregiao,id_estado
0,1102,Leste Rondoniense,11
1,1102,Leste Rondoniense,11
2,1102,Leste Rondoniense,11
3,1102,Leste Rondoniense,11
4,1102,Leste Rondoniense,11


In [15]:
# DataFrame das MicroRegioes
df_microregioes = pd.DataFrame(ls_microregiao,
                               columns = ['id','nome','id_mesoregiao'])
df_microregioes.rename(columns = {'id' : 'id_microregiao',
                                 'nome': 'nome_microregiao'},
                                 inplace = True)

In [16]:
# DataFrame Auxiliar com os municipios
df_municipios_aux = pd.DataFrame(municipios, 
                          columns = ['id', 'nome', 'id_microrregiao'])
df_municipios_aux.rename(columns = { 'id' : 'id_municipio',
                                     'nome': 'nome_municipio',
                                     'sigla': 'sigla_municipio'},
                             inplace = True)

In [17]:
# Gerando o DataFrame com as informações dos Municipios
df_municipios = pd.concat([df_mesoregioes, df_microregioes, df_municipios_aux], 
                       axis = 1)

In [18]:
df_municipios.head(3)

,id_mesoregiao,nome_mesoregiao,id_estado,id_microregiao,nome_microregiao,id_mesoregiao,id_municipio,nome_municipio,id_microrregiao
0,1102,Leste Rondoniense,11,11006,Cacoal,1102,1100015,Alta Floresta D'Oeste,11006
1,1102,Leste Rondoniense,11,11003,Ariquemes,1102,1100023,Ariquemes,11003
2,1102,Leste Rondoniense,11,11008,Colorado do Oeste,1102,1100031,Cabixi,11008


## Gerando os Arquivos .CSVs

Uma vez feito o processo de preparação dos dados e de montagem dos dataframes, será criado os arquivos .CSVs para que possam ser utilizados nas mais variadas aplicações envolvendo informações relativas a federação.

### Criar a pasta onde ficara os arquivos 

In [43]:
# Captar o Diretório atual
path = os.getcwd()

In [52]:
# Criar uma pasta no diretorio atual, para conter os arquivos .CSVs
pasta_arquivos_csv = path + "/data"    
retorno, mensagem = fn_criar_pasta(pasta_arquivos_csv)
mensagem

'O diretório /home/joao-santos/Projetos/Python/Aplicacoes/IBGE/data foi criado com sucesso.'

#### Exportando Estados

In [58]:
estados_csv = pasta_arquivos_csv + "/estados.csv"
export_est_csv = df_estados.to_csv (estados_csv, index = None, header=True) 

#### Exportando Municipios

In [59]:
municipios_csv = pasta_arquivos_csv + "/municipios.csv"
export_muni_csv = df_municipios.to_csv (municipios_csv, index = None, header=True) 